In [7]:
# Instalación de embedder (Gemini) y un almacenador de vectores (Chroma)
!pip install -U -q google-generativeai
!pip install -q chromadb
!pip install -q openai

In [9]:
import textwrap
import numpy as np
import pandas as pd
from IPython.display import Markdown
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings

from google.colab import userdata
import os
import google.generativeai as genai

# Gemini API
API_KEY=userdata.get('API_KEY_GEMINI')
genai.configure(api_key=API_KEY)

In [10]:
# Importar data
data = pd.read_excel('/content/dataset_final.xlsx')
data.head()

,Categoría,Pregunta,Respuesta
0,Banca,¿Cómo cambio mi clave de acceso a la banca en ...,Accede a 'Seguridad' en tu cuenta y redefine t...
1,Banca,¿Cómo habilito mi tarjeta para compras interna...,Activa compras internacionales desde la app; ¡...
2,Seguros,¿Cómo activo la cobertura inmediata en mi segu...,Activa la cobertura inmediata llamando a tu as...
3,Banca,¿Cómo activo mi tarjeta de débito?,Actívala directamente en un cajero automático ...
4,Seguros,¿Cómo actualizo mi dirección en mi seguro de h...,Actualiza tu dirección en la configuración de ...


In [11]:
# Estructura de datos a vectorizar
data['Pregunta_respuesta'] = data['Pregunta'] + " " + data['Respuesta']
data_text = data['Pregunta_respuesta']
data_text.head()

,Pregunta_respuesta
0,¿Cómo cambio mi clave de acceso a la banca en ...
1,¿Cómo habilito mi tarjeta para compras interna...
2,¿Cómo activo la cobertura inmediata en mi segu...
3,¿Cómo activo mi tarjeta de débito? Actívala di...
4,¿Cómo actualizo mi dirección en mi seguro de h...


In [26]:
# Convertir la columna "Pregunta_Respuesta" en una lista de documentos.
documents = data["Pregunta_respuesta"] # Access the column from the original DataFrame 'data'

# Verificar los primeros documentos
for i, doc in enumerate(documents):  # Mostrar con límie documents[:x], donde x:n° de documentos a visualizar.
    print(f"DOCUMENT{i} = {repr(doc)}\n")

DOCUMENT0 = "¿Cómo cambio mi clave de acceso a la banca en línea? Accede a 'Seguridad' en tu cuenta y redefine tu clave en minutos."

DOCUMENT1 = '¿Cómo habilito mi tarjeta para compras internacionales? Activa compras internacionales desde la app; ¡prepárate para tu próximo viaje!'

DOCUMENT2 = '¿Cómo activo la cobertura inmediata en mi seguro de auto? Activa la cobertura inmediata llamando a tu aseguradora desde el primer día.'

DOCUMENT3 = '¿Cómo activo mi tarjeta de débito? Actívala directamente en un cajero automático o mediante tu aplicación móvil.'

DOCUMENT4 = '¿Cómo actualizo mi dirección en mi seguro de hogar? Actualiza tu dirección en la configuración de tu cuenta bancaria.'

DOCUMENT5 = "¿Cómo puedo cambiar mi dirección registrada en el banco? Actualiza tu dirección fácilmente desde el apartado de 'Perfil' en tu banca digital."

DOCUMENT6 = "¿Cómo modifico los beneficiarios de mi seguro de vida? Actualiza tus beneficiarios en 'Configuración de seguros' dentro de tu banca en 

# API Keys

In [13]:
import openai
import os

# Configurar la clave de API de OpenAI
openai.api_key = '###################' # Anonimizada
os.environ["OPENAI_API_KEY"] = openai.api_key

openai.api_key = os.environ.get('OPENAI_API_KEY')
client = openai.OpenAI()
answer = client.chat.completions.create

In [14]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

def get_embedding(text, model="text-embedding-ada-002"):
    """
    Genera un embedding vectorial para un texto utilizando un modelo de embeddings.

    Args:
        text (str): Texto de entrada para el cual se generará el embedding.
        model (str): Nombre del modelo de OpenAI utilizado para generar el embedding.
                     Por defecto, utiliza el modelo "text-embedding-ada-002".

    Returns:
        list: Un vector (embedding) de alta dimensionalidad que representa la semántica del texto.
    """

    text = text.replace("\n", " ")

    response = client.embeddings.create(input=[text], model=model)

    return response.data[0].embedding

embedding_model = "text-embedding-ada-002"

# Ensure 'documents' is a DataFrame or Series before applying the function
documents = data["Pregunta_respuesta"]  # This assumes 'data' is your DataFrame
documents = documents.to_frame()  # If 'documents' is a Series, convert it to a DataFrame
documents["embedding"] = documents['Pregunta_respuesta'].apply(lambda x: get_embedding(x, model=embedding_model))

# Ensure there are embeddings to concatenate
if len(documents.embedding.values) > 0:
    matrix = np.vstack(documents.embedding.values)
else:
    matrix = np.array([])

In [15]:
documents.head()

,Pregunta_respuesta,embedding
0,¿Cómo cambio mi clave de acceso a la banca en ...,"[-0.015041896142065525, -0.022150302305817604,..."
1,¿Cómo habilito mi tarjeta para compras interna...,"[-0.004591276869177818, -0.003662846749648452,..."
2,¿Cómo activo la cobertura inmediata en mi segu...,"[-0.01625330187380314, -0.010508600622415543, ..."
3,¿Cómo activo mi tarjeta de débito? Actívala di...,"[-0.030755281448364258, -0.010065251030027866,..."
4,¿Cómo actualizo mi dirección en mi seguro de h...,"[-0.017373504117131233, -0.0004793013795278966..."


In [12]:
# Exportar embeddings
documents.to_excel('documents.xlsx')

In [19]:
import ast
import chromadb
from chromadb.config import Settings
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

# Crear la base de datos Chroma
def create_chroma_db(documents, collection_name):
    """
    Crea una base de datos vectorial en Chroma a partir de un conjunto de documentos y sus embeddings.

    Args:
        documents (pandas.DataFrame): Un DataFrame que contiene los documentos y sus embeddings precomputados.
                                      Debe incluir las columnas:
                                      - 'Pregunta_respuesta': Texto unificado del documento.
                                      - 'embedding': Embedding asociado al documento.
        collection_name (str): Nombre de la colección que se creará en ChromaDB.

    Returns:
        chromadb.Collection: La colección creada en ChromaDB que contiene los documentos y sus embeddings.

    """
    chroma_client = chromadb.Client()
    embedding_function = OpenAIEmbeddingFunction(api_key=openai.api_key)  # Usar la API de OpenAI
    db = chroma_client.create_collection(collection_name, embedding_function=embedding_function)

    # Agregar los documentos y embeddings a la colección
    for i, row in documents.iterrows():
        # Check if the embedding is already a list
        if isinstance(row['embedding'], list):
            embedding = row['embedding']
        else:
            # If not a list, try to convert it using ast.literal_eval
            try:
                embedding = ast.literal_eval(row['embedding'])
            except (SyntaxError, ValueError):
                # If conversion fails, print a warning and skip the row
                print(f"Warning: Could not convert embedding for row {i}. Skipping.")
                continue
        db.add(
            documents=[row["Pregunta_respuesta"]],
            embeddings=[embedding],  # Embeddings precomputados
            ids=[str(i)]
        )

    print(f"Chroma collection '{collection_name}' created with {len(documents)} documents.")
    return db

In [17]:
# Función para realizar consultas en la base de datos Chroma
def query_chroma(query, db, n_results=1):
    """
    Realiza una consulta a la base de datos Chroma para encontrar el texto más relevante.

    Args:
        query (str): Texto de consulta.
        db (chromadb.Collection): Colección de Chroma donde buscar.
        n_results (int): Número de resultados más relevantes a devolver.

    Returns:
        list: Lista de los textos más relevantes encontrados.
    """
    results = db.query(query_texts=[query], n_results=n_results)
    return results["documents"][0] if "documents" in results and results["documents"] else []


In [21]:
# Crear la base de datos en Chroma
chroma_collection = create_chroma_db(documents,"prueba.autogestion")

Chroma collection 'prueba.autogestion' created with 1041 documents.


In [22]:
# Consultar la base de datos con un query
query = "¿Cómo consultar el saldo de mi cuenta?"
relevant_passages = query_chroma(query, chroma_collection, n_results=1)

# Mostrar los resultados
print("Relevant passage(s):")
for passage in relevant_passages:
  print(passage)

Relevant passage(s):
¿Cómo puedo consultar el saldo de mi cuenta? Puedes consultar el saldo de tu cuenta a través de la banca en línea, la aplicación móvil o en un cajero automático.


# Integración LLM

In [23]:
def query_prompt(query, relevant_passage):
    """
    Crea un prompt para el LLM utilizando la consulta del usuario y un documento relevante.

    Args:
        query (str): Consulta realizada por el usuario.
        relevant_passage (str): Documento relevante recuperado de la base de datos.

    Returns:
        str: Prompt contextualizado para el LLM.
    """
    # Limpieza básica del documento relevante
    escaped_passage = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")

    # Construcción del prompt
    prompt = (f"""
    Eres un asistente útil e informativo que responde preguntas basándote en el texto proporcionado en el pasaje de referencia incluido a continuación.
    Asegúrate de responder de manera completa, siendo comprensivo y proporcionando toda la información de contexto relevante.
    Sin embargo, estás hablando con una audiencia no técnica, así que desglosa los conceptos complicados y utiliza recursos amigable.
    Si la pregunta no es relevante para la respuesta, puedes derivar la respuesta hacia un ejecutivo u oficina de atención.

    PREGUNTA: '{query}'
    PASAJE: '{escaped_passage}'

    RESPUESTA:
    """)
    return prompt

In [24]:
def generate_response(query, db, model):
    """
    Genera una respuesta personalizada utilizando el LLM y un documento relevante de ChromaDB.

    Args:
        query (str): Consulta realizada por el usuario.
        db: Objeto de la colección ChromaDB.
        model: Objeto del modelo generativo (e.g., Gemini o GPT).

    Returns:
        str: Respuesta generada por el LLM.
    """
    # Recuperar el pasaje relevante desde ChromaDB
    relevant_passage = db.query(query_texts=[query], n_results=1)['documents'][0][0]

    # Crear el prompt contextualizado
    prompt = query_prompt(query, relevant_passage)

    # Generar la respuesta utilizando el modelo generativo
    response = model.generate_content(prompt)

    return response.text

In [25]:
# Simular una consulta del usuario
query = "¿Cómo puedo consultar mi saldo bancario?"

# Obtener el modelo generativo (Gemini, GPT, etc.)
model = genai.GenerativeModel('gemini-1.5-flash-latest')

# Generar la respuesta
response = generate_response(query, chroma_collection, model)

# Mostrar la respuesta generada
print("Respuesta del LLM:")
print(response)

Respuesta del LLM:
Para consultar tu saldo bancario, tienes dos opciones muy sencillas:

1. **A través de la aplicación móvil de tu banco:**  Si tu banco tiene una aplicación para teléfonos inteligentes (como un celular o tablet),  descarga la app y regístrate.  Normalmente, necesitarás tu número de cuenta y alguna otra información de seguridad (como una contraseña o un PIN).  Una vez que inicies sesión,  tu saldo debería aparecer inmediatamente en la pantalla principal.  Es rápido, fácil y puedes hacerlo desde cualquier lugar con conexión a internet.

2. **En un cajero automático (ATM):**  Visita cualquier cajero automático de tu banco. Introduce tu tarjeta de débito y tu PIN.  Generalmente, hay una opción en el menú principal para consultar tu saldo.  Selecciona esa opción y el cajero automático te mostrará tu saldo actual en la pantalla.  Esta opción es útil si no tienes acceso a internet o prefieres no usar tu teléfono móvil.


Si tienes problemas con cualquiera de estos métodos, t